<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroConsumer
from confluent_kafka.avro.serializer import SerializerError
import json
import qcutils

topic = ''
consumer_group = ''

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))
sr_url=qcutils.read_config_value("kafka.schema_registry.url")

**Note**: in order to avoid conflicts during read operation, please name:
* the topic as `<surname>-avro-card`
* the consumer group as follow `<surname>-cg`

# write a kafka avro Consumer

Write an avro consumer that consumes the avro topic your produced in Lab 3

In [ ]:
consumerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url,
        'group.id': consumer_group,
        'auto.offset.reset': 'earliest'
    }

c = AvroConsumer(consumerconf)

# Subscribe to topic
c.subscribe([topic])

waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = msg.value()                                  # get the value of the message
            key = msg.key()                                      # get the key of the message
            print("\nConsumed record with key {} and value {}"
                  .format(key, value['year']))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()  

### Final Note
To access the `year` you can directly ask to the deserialized message.

Using Avro, you can control the input message. 

A producer can only send message that respect the schema.

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.